In [1]:
import sys;print(sys.version)

3.10.8 (v3.10.8:aaaf517424, Oct 11 2022, 10:14:40) [Clang 13.0.0 (clang-1300.0.29.30)]


In [2]:
os.getcwd() 

'/Users/biancaziliotto/Desktop/GitRepo/MLx2'

In [1]:
os.chdir('src')

In [2]:
from MLP import MLP 

In [3]:
mlp = MLP(0, [], 3, 2)

In [9]:
mlp.layers

In [8]:
mlp = MLP(1, [1], 3, 2)

TypeError: Layer.__init__() takes 1 positional argument but 4 were given

In [10]:
import numpy as np
X = np.ndarray((4,3))
print(X)
mlp.predict(X)

[[-3.10503618e+231  1.73060555e-077  2.96439388e-323]
 [ 0.00000000e+000  4.31476307e+069  1.04082753e-047]
 [ 5.28228494e-091  5.98633921e+174  5.45238584e-067]
 [ 1.02255265e+166  3.99910963e+252  8.34404934e-309]]


NameError: name 'layer_input' is not defined